In [44]:
import requests as r
import pandas as pd 
from bs4 import BeautifulSoup as bs
import re
import time as t
from requests import Session
import json
import datetime

In [45]:
pages = 0
url = 'https://www.google.com/search?q=bitcoin&client=firefox-b-1-d&tbm=nws&ei=jWIWXYnSDorZ5gLo8JbIDg&start='+str(pages)+'&sa=N&ved=0ahUKEwjJzJ6K7YzjAhWKrFkKHWi4Bek4WhDy0wMIXA&biw=1320&bih=737&dpr=1.09'


In [52]:
def request(url,start=0, pages=1,): 
    
    url = 'https://www.google.com/search?q=bitcoin&client=firefox-b-1-d&tbm=nws&ei=jWIWXYnSDorZ5gLo8JbIDg&start='+str(pages)+'&sa=N&ved=0ahUKEwjJzJ6K7YzjAhWKrFkKHWi4Bek4WhDy0wMIXA&biw=1320&bih=737&dpr=1.09'
    soup_string_list = []
    soup_list = []
    if pages != 1:
        
        try:
            
            for i in range(start,pages):
                'Page %s in progress...'%(i)

                pages += (i*10)-10
                html = r.get(url, timeout=1).content
                soup = bs(html, 'lxml')

                soup_string = str(bs(html, 'lxml'))
                soup_string_list.append(soup_string)
                soup_string = ''.join(soup_string_list)

                soup_list.append(soup) 
                
                
                
                t.sleep(1)
            return soup_string, soup_list
        except: 
            pass
        
    else:
        html = r.get(url, timeout=1).content
        soup_string = str(bs(html, 'lxml'))
        
        soup = bs(html, 'html.parser')
        soup.prettify()
        
        return soup_string, soup
    
    
    
def clean_titles(soup_string):
    
    search = re.findall('">.{0,200}</a></h3>', soup_string)
    sub_1 = list(map(lambda x: re.sub(r"(</b>)",r"",x), search))
    sub_2 = list(map(lambda x: re.sub(r"(<b>)",r"",x), sub_1))
    rstrip = list(map(lambda x: x.strip('</a></h3>'), sub_2))
    lstrip = list(map(lambda x: x.lstrip('">'), rstrip))
    
    return lstrip


def clean_sources(soup): 
    clean = []
    
    for i in soup:
        search = i.find_all('div', attrs={'class':'slp'})
        text = [i.text for i in search]
        split = list(map(lambda x: x.split('-'), text))
        rstrip = list(map(lambda x: x[0].rstrip(), split))
        
        clean.append(rstrip)
        
    flatten = [i for mid in clean for i in mid]
    
    return flatten



def clean_dates(soup): 
    clean = []
    
    for i in soup: 
        search = i.find_all('div', attrs={'class':'slp'})
        text = [i.text for i in search]
        split = list(map(lambda x: x.split('-'), text))
        rstrip = list(map(lambda x: x[1].rstrip(), split))
        lstrip = list(map(lambda x : x.lstrip(), rstrip))
        
        clean.append(lstrip)
        
    flatten = [i for mid in clean for i in mid]
    
    return flatten














In [49]:
%%time
titles, DatesAndSources = request(url,0,300)

CPU times: user 13.5 s, sys: 1.72 s, total: 15.2 s
Wall time: 22min 41s


In [50]:
print('Titles Length:%s\nSources Length:%s\nDates Length:%s'%(len(clean_titles(titles)),
                                                              len(clean_sources(DatesAndSources)),
                                                             len(clean_dates(DatesAndSources))))

Titles Length:0
Sources Length:0
Dates Length:0


In [19]:
df = pd.DataFrame()

In [20]:
df['Title'] = clean_titles(titles)
df['Source'] = clean_sources(DatesAndSources)
df['Date'] = clean_dates(DatesAndSources)
df[df['Date']=='3 days ago']
df.head()

,Title,Source,Date
0,Bitcoin Climbs to One-Year High on Facebook Cr...,Bloomberg,"Jun 16, 2019"
1,Facebook's Cryptocurrency Could Be Good News f...,Barron's,"Jun 14, 2019"
2,Bitcoin Hits 15-Month High as Naysayers Double...,Cointelegraph,5 days ago
3,$200000 in Bitcoin Seized in Dark Net Drug Probe,CoinDesk,"Jun 18, 2019"
4,Bitcoin Struggles for Price Gains As Litecoin ...,CoinDesk,"Jun 12, 2019"


In [23]:
df_daily.shape

(1025, 3)

In [22]:
df_daily = df[df['Date'].str.contains('2019|2018')]

In [24]:
def format_date(series=df_daily['Date']):

    unformated = list(series)
    formated = []
    try: 
        for i in unformated: 
            if '2019' in i:  

                take = datetime.datetime.strptime(i, '%b %d, %Y')
                cut = take.isoformat()[:10]
                formated.append(cut)
    except:
        formated.append(i)

    return formated



def BTC(start='2019-01-01', end='2019-07-25'):  
    
    key = 'fcf1f334-f9b2-4acd-8024-3b425c1272a4'
    domain = 'https://api.coindesk.com/v1/bpi/historical/close.json?start=%s&end=%s'%(start,end)
    JSON = r.get(domain).json()['bpi']
    keys = list(JSON)
    
    df = pd.DataFrame()
    df['Date'] = keys
    
    values = []
    for i in keys: 
        values.append(JSON[i])
        
    df['Price'] = values
    
    return df

In [25]:
print(len(df_daily['Date']))
print(len(format_date()))

1025
1025


In [26]:
df_daily['Date'] = format_date()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
df_daily.head()

,Title,Source,Date
0,Bitcoin Climbs to One-Year High on Facebook Cr...,Bloomberg,2019-06-16
1,Facebook's Cryptocurrency Could Be Good News f...,Barron's,2019-06-14
3,$200000 in Bitcoin Seized in Dark Net Drug Probe,CoinDesk,2019-06-18
4,Bitcoin Struggles for Price Gains As Litecoin ...,CoinDesk,2019-06-12
6,Bitcoin Price Eyes Stronger Recovery Rally Aft...,CoinDesk,2019-06-07


In [28]:
mini = df_daily['Date'].min()
maxi = df_daily['Date'].max()


In [29]:
join_df = BTC(mini, maxi).join(df_daily.set_index('Date'),on='Date')

In [30]:
join_df.shape

(1033, 4)

In [31]:
unique_df = join_df.drop_duplicates('Title').reset_index()

In [32]:
unique_df

,index,Date,Price,Title,Source
0,0,2019-06-05,7785.9850,Apple's iPhone Is Taking Its First Steps Into ...,Forbes
1,1,2019-06-06,7806.7150,NaN,NaN
2,2,2019-06-07,8001.2767,Bitcoin Price Eyes Stronger Recovery Rally Aft...,CoinDesk
3,5,2019-06-10,8016.6917,Michael Ford Named Newest Bitcoin Core Code Ma...,CoinDesk
4,5,2019-06-10,8016.6917,Crypto Platform Says It Aims to Solve Bitcoin ...,Cointelegraph
5,5,2019-06-10,8016.6917,Bitcoin Risks Short-Term Bear Reversal Below $...,CoinDesk
6,7,2019-06-12,8173.3567,Bitcoin Struggles for Price Gains As Litecoin ...,CoinDesk
7,9,2019-06-14,8693.7650,Facebook's Cryptocurrency Could Be Good News f...,Barron's
8,11,2019-06-16,8976.5367,Bitcoin Climbs to One-Year High on Facebook Cr...,Bloomberg
9,11,2019-06-16,8976.5367,Bitcoin tops $9000 level for first time in 13 ...,MarketWatch


pandas.core.frame.DataFrame